In [8]:
import pandas as pd 
from utils import extract_colors_image,extract_colors_url

In [3]:
url=extract_colors_url("https://nestorevintage.fr/cdn/shop/files/307FDA5D-D3FB-48D6-8DA2-2478DF104BD3.jpg?v=1738348432&width=2048")

c:\Users\BahaARFAOUI\OneDrive - CBTW\Bureau\colorfact\colorfact\utils.py:80: RuntimeWarning: overflow encountered in scalar subtract
  a_std = float(a_opencv - 128)


In [4]:
url.colors

[[58.03921568627452, 251.0, 26.0], [46.27450980392157, 253.0, 23.0]]

In [4]:
image=extract_colors_image(image_path="data/doudoune.png")

In [5]:
image.colors

[[40.3921568627451, -7.0, 6.0], [23.52941176470588, -7.0, 6.0]]

In [6]:
im=image.remove_background_grabcut()

In [9]:
import pandas as pd
data=pd.read_excel("data/final_data.xlsx")

In [10]:
data=data.drop("Unnamed: 0",axis=1)

In [13]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm

def process_image(image_url):
    """Extracts dominant colors from an image URL."""
    try:
        color_extractor = extract_colors_url(image_url)
        return color_extractor.colors
    except Exception as e:
        print(f"Error processing {image_url}: {str(e)}")
        return None

def process_dataset(df: pd.DataFrame, num_workers=8) -> pd.DataFrame:
    """
    Processes a dataset containing image links and adds a 'cielab_colors' column
    with the dominant colors extracted from each image using multithreading.

    Args:
        df (pd.DataFrame): Input dataframe containing a 'Photo produit' column.
        num_workers (int): Number of threads for parallel processing.

    Returns:
        pd.DataFrame: Modified dataframe with 'cielab_colors' column added.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    if 'Photo produit 1' not in df.columns:
        raise ValueError("DataFrame must contain a 'Photo produit' column")

    image_urls = df['Photo produit 1'].tolist()

    # Process images in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(executor.map(process_image, image_urls), total=len(image_urls)))

    df['cielab_colors'] = results
    return df

# Example usage:
#df_processed = process_dataset(df_corrected, num_workers=8)


In [19]:
df_processed_2=data.iloc[700:800,:]

In [20]:
df_processed_2_pro = process_dataset(df_processed_2, num_workers=8)


100%|██████████| 100/100 [1:12:18<00:00, 43.38s/it]


In [21]:
df_processed_2_pro.to_excel("data/data_700_800.xlsx")